In [1]:
import os
import subprocess
import shutil

import pandas as pd
from tqdm import tqdm

In [2]:
problem_name = "MiniGroceryStore"

In [3]:
PROBLEM_DIR = "problems"
CODES_DIR = os.path.join(PROBLEM_DIR, problem_name, "codes")
TESTCASES_DIR = os.path.join(PROBLEM_DIR, problem_name, "testcases")
SUBMISSIONS_DIR = os.path.join(PROBLEM_DIR, problem_name, "submissions")

In [4]:
for path in [TESTCASES_DIR, CODES_DIR]:
    if not os.path.exists(path):
        raise FileNotFoundError(path)

In [5]:
java_list = [ path for path in os.listdir(CODES_DIR) if path.endswith(".java") ]

In [6]:
for filename in java_list:
    sid, name = filename.split(".", 1)
    if name != f"{problem_name}.java":
        raise ValueError(filename)

In [7]:
def read_case(filename: str) -> str:
    with open(filename, "r") as fp:
        return fp.read().strip()

In [8]:
infile_list = sorted([path for path in os.listdir(TESTCASES_DIR) if path.endswith(".in")], key=lambda x: int(x.split(".")[0]))
solfile_list = sorted([path for path in os.listdir(TESTCASES_DIR) if path.endswith(".sol")], key=lambda x: int(x.split(".")[0]))


testcase_list = [
    (
        read_case(os.path.join(TESTCASES_DIR, infile)),
        read_case(os.path.join(TESTCASES_DIR, solfile)),
    )
    for infile, solfile in zip(infile_list, solfile_list)
]

In [9]:
def compile_java(dir: str, filename: str) -> int:
    cmd = f"javac {os.path.join(dir, filename)} -d {dir}"
    status = os.system(cmd)
    # if status != 0:
    #     raise Exception(f"Compilation failed with status = {status} (cmd={cmd})")
    return status

def run(dir: str, name: str, input: int) -> str:
    cmd = f"java {name}"
    p = subprocess.Popen(
        cmd,
        stdin=subprocess.PIPE,
        stdout=subprocess.PIPE,
        cwd=dir,
    )
    stdout, stderr = p.communicate(input=str.encode(input))
    output = stdout.decode()
    return output

def clean(dir):
    """Remove all files ending with '.class'"""
    for root, dirs, files in os.walk(dir):
        for file in files:
            if file.endswith(".class"):
                filepath = os.path.join(root, file)
                os.remove(filepath)

In [10]:
def compare_output(label: str, output: str) -> bool:
    label = label.strip().splitlines()
    output = output.strip().splitlines()

    if len(label) != len(output):
        return False

    for l, o in zip(label, output):
        if l.strip() != o.strip():
            return False

    return True

In [11]:
if os.path.exists(SUBMISSIONS_DIR):
    print("REMOVE:", SUBMISSIONS_DIR)
    shutil.rmtree(SUBMISSIONS_DIR)
    
os.mkdir(SUBMISSIONS_DIR)
print("CEATED:", SUBMISSIONS_DIR)

data = []
for filename in tqdm(java_list):
    # create dir
    sid, name = filename.split(".", 1)
    dir_ = os.path.join(SUBMISSIONS_DIR, sid)
    os.mkdir(dir_)

    # # copy java
    src = os.path.join(CODES_DIR, filename)
    dst = os.path.join(dir_, name)
    shutil.copy2(src, dst)

    # compile
    status = compile_java(dir_, name)

    # run
    item = {
        "student_id": sid,
    }
    for i, (input, sol) in enumerate(testcase_list, 1):
        output = run(dir_, problem_name, input)
        output = "\n".join(output.splitlines()).strip()
        with open(os.path.join(dir_, f"{i}.out"), "w") as fp:
            fp.write(output)
        score = 1 if compare_output(output, sol) else 0
        item[f"{i}"] = score
    data.append(item)

CEATED: problems\MiniGroceryStore\submissions


100%|██████████| 91/91 [01:49<00:00,  1.21s/it]


In [12]:
df = pd.DataFrame(data)
df

,student_id,1,2,3,4,5,6,7
0,620710733,1,1,1,1,0,0,0
1,630710836,0,0,0,0,0,0,0
2,640710546,0,0,0,0,0,0,0
3,650710151,0,0,0,0,0,0,0
4,650710157,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
86,670710743,1,1,1,1,0,0,0
87,670710746,0,0,0,0,0,0,0
88,670710977,0,0,0,0,0,0,0
89,670710978,0,0,0,0,0,0,0


In [13]:
df["sum"] = df[[ f"{i}" for i in range(1, len(testcase_list)+1) ]].sum(axis=1)
df

,student_id,1,2,3,4,5,6,7,sum
0,620710733,1,1,1,1,0,0,0,4
1,630710836,0,0,0,0,0,0,0,0
2,640710546,0,0,0,0,0,0,0,0
3,650710151,0,0,0,0,0,0,0,0
4,650710157,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
86,670710743,1,1,1,1,0,0,0,4
87,670710746,0,0,0,0,0,0,0,0
88,670710977,0,0,0,0,0,0,0,0
89,670710978,0,0,0,0,0,0,0,0


In [14]:
df.to_csv(os.path.join(PROBLEM_DIR, problem_name, f"{problem_name}.csv"), index=None)